<a href="https://colab.research.google.com/github/sandynayak007/Coursera_Capstone/blob/main/Final_submission_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Where to open the most popular Cocktail bar in the town?
### Introduction

In the gastro environment bars of all sorts play a major role. From a simple pub serving beer and maybe some food to exclusive cocktail bars the array and style is unique to every venue. While the venue itself is of great importance to maintaining a solid customers basis, finding new customers can be greatly influenced by the location and neighborhood.
This study focuses on a regional study within Delhi to open a cocktail bar. No food will be served. While the venue wants to capture a high number of high-end customers who will travel to the bar specifically, the goal is to grow their customer base based on location. Essentially making it more accessible and favorable to a broader audience. All business aspects like pricing and products on offer are neglected, focusing solely on the most favorable location to supplement the venue while still making it a standout establishment.
As the bar will not serve any food and be a unique venue for one or two drinks in the afternoon/evening following informations were taken into consideration:

·       Proximity to an array of restaurants

·       Reduced competition in the region

### Data

Using the Foursquare API the locations of restaurants, bars, train-, tram stations and bus stops were selected. Delhi is made up of 11 district hence 11 clusters of restaurants, public transport and rival bars were created. The clusters center would serve as the most favourable or least favourable location. Based on the data it might be possible to view regions most fruitful to open the cocktail bar at. Optimally regions with high densities of restaurants and public tranpsort and low densities of bars would be recorded. To also view the accessiblity for an after work drink public transport point (train, tram and bus stations) were clustered.

In [98]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
#from sklearn.datasets.samples_generator import make_blobs

!pip install bs4
from bs4 import BeautifulSoup
import requests

from geopy.geocoders import Nominatim 
import requests
! conda install -c conda-forge folium
#import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from collections import OrderedDict
import matplotlib as mpl

from pandas.io.json import json_normalize
import folium

import json

print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


In [99]:
CLIENT_ID = 'BKCBJY14KSTRO2IXOHRRX3S15LZY5FEP0S1NJ3GCDC3TIJS1' # your Foursquare ID
CLIENT_SECRET = 'P1134OU5DYLOOOZ3J5NJPZGT1IHDPXVUGQKUY4PYWXYJYOEW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
Limit = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BKCBJY14KSTRO2IXOHRRX3S15LZY5FEP0S1NJ3GCDC3TIJS1
CLIENT_SECRET:P1134OU5DYLOOOZ3J5NJPZGT1IHDPXVUGQKUY4PYWXYJYOEW


In [100]:
#other search parameters
city = 'Delhi'

In [101]:
#funtion to search for a query near a city, returns json file
def SearchVenues(query):
    
    search_results=[]
    for searchtopic in zip(query):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            city, 
            Limit,
            query)
        results = requests.get(url).json()['response']['groups'][0]['items']
    
        search_results.append([(
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng']) for v in results])
    
        return (search_results)

In [102]:
#creating json file with wanted content
query1 = 'restaurant'
restaurants = SearchVenues(query1)
restaurants

[[('Thai High', 28.522127097129164, 77.1818314461584),
  ('Kainoosh', 28.54263074146378, 77.15688562351326),
  ('Olive Bar & Kitchen', 28.526103185630244, 77.1839758878834),
  ('The Big Chill Cafe', 28.528201144658645, 77.21774809752226),
  ('Maxims', 28.552849276682906, 77.24206544536278),
  ("L'Opera", 28.599784379249712, 77.2261441150033),
  ('The Big Chill Cafe', 28.552727889814417, 77.24192322690806),
  ('Indian Accent', 28.592025841987475, 77.23800557290183),
  ('Hawkers', 28.523172143706116, 77.15736168796424),
  ('Jamavar Kashmiri Restaurant @ The Leela',
   28.580142785889475,
   77.18926702333043),
  ('Big Yellow Door', 28.693244735370346, 77.20494771461874),
  ('Varq | वर्क', 28.60454694756125, 77.2237810008916),
  ('The Ploof Deli Marketplace', 28.58358302989589, 77.22328810710694),
  ('Amour Bistro', 28.60156938620385, 77.1859231921711),
  ('Tamra', 28.620543184230115, 77.21817445898111),
  ('Bukhara', 28.596914259545567, 77.17335846829526),
  ('Khan Chacha', 28.5281703835

In [103]:
query2 = 'drinks'
bars = SearchVenues(query2)
bars

[[('The Lodhi Hotel', 28.591668747194344, 77.2381313007319),
  ('PCO', 28.561635344367186, 77.1555896199453),
  ("Lantern's", 28.64318296013676, 77.17774563974753),
  ('On The Waterfront', 28.592061411987267, 77.23795124764315),
  ('Blue Bar', 28.59540260693495, 77.17052851368256),
  ('Public Affair', 28.600030292398554, 77.22647823218283),
  ('SODABOTTLEOPENERWALA', 28.600140869804918, 77.22627303643019),
  ('b-bar', 28.528592815547366, 77.21834124754761),
  ('Warehouse Cafe', 28.633775293123456, 77.22101540430111),
  ('Monkey Bar (Monkey Bar New Delhi)', 28.53657802581153, 77.14766875358471),
  ('Shalom', 28.556492280624543, 77.23347659326305),
  ('Lighthouse 13', 28.529996813332993, 77.2198035499783),
  ('Cheri One', 28.526809865633457, 77.18576940640662),
  ('Ricks', 28.60459984359947, 77.22371413090666),
  ('Nehru Place Social', 28.551309797156915, 77.25110298208138),
  ('Library Bar', 28.580116202772658, 77.18979479443512),
  ("Q'La", 28.527280703140192, 77.18579521803618),
  ('O

In [104]:
query3 = 'metro station'
metro = SearchVenues(query3)
metro

[[('Delhi Aerocity Metro Station', 28.54857229090808, 77.12087559195506),
  ('Dwarka Metro Station', 28.614938122598893, 77.02277541160583),
  ('New Delhi | नई दिल्ली Metro Station', 28.64280345814337, 77.2233510017395),
  ('Patel Chowk Metro Station', 28.622511375939013, 77.21367934985605),
  ('Indira Gandhi International Airport Metro Station',
   28.555696368301586,
   77.08622342663494),
  ('Central Secretariat Metro Station', 28.61590821291491, 77.21255779266357),
  ('Jhilmil Metro Station', 28.675875696132426, 77.31230871219229),
  ('Janak Puri East Metro Station', 28.632935135027907, 77.08675146102905),
  ('Airport Express Line Metro Station',
   28.549346923827297,
   77.12086915969624),
  ('Jama Masjid Metro Station', 28.649631, 77.237812),
  ('Pulbangsh metro station', 28.66641586680745, 77.2074294090271),
  ('South ex', 28.559267044067383, 77.2186050415039),
  ('Kashmere Gate Metro Station', 28.667746149207634, 77.22830359986663),
  ('Khan Market Metro Station', 28.602284745

In [105]:

#writting json data into panda df
df_rest = pd.DataFrame(item for venue_list in restaurants for item in venue_list)
df_bars = pd.DataFrame(item for venue_list in bars for item in venue_list)
df_metro=pd.DataFrame(item for venue_list in metro for item in venue_list)
df_metro.head(5)

,0,1,2
0,Delhi Aerocity Metro Station,28.548572,77.120876
1,Dwarka Metro Station,28.614938,77.022775
2,New Delhi | नई दिल्ली Metro Station,28.642803,77.223351
3,Patel Chowk Metro Station,28.622511,77.213679
4,Indira Gandhi International Airport Metro Station,28.555696,77.086223


In [106]:
#view shape
print(df_rest.shape)
print(df_bars.shape)
print(df_metro.shape)

#edit column titles
df_rest.columns = ('Name', 'Lat', 'Lng')
df_bars.columns = ('Name', 'Lat', 'Lng')
df_metro.columns=('Name', 'Lat', 'Lng')

(99, 3)
(99, 3)
(58, 3)


In [107]:
print("Metro-Station information")
df_metro.head()

Metro-Station information


,Name,Lat,Lng
0,Delhi Aerocity Metro Station,28.548572,77.120876
1,Dwarka Metro Station,28.614938,77.022775
2,New Delhi | नई दिल्ली Metro Station,28.642803,77.223351
3,Patel Chowk Metro Station,28.622511,77.213679
4,Indira Gandhi International Airport Metro Station,28.555696,77.086223


In [108]:
print("Bar information")
df_bars.head()

Bar information


,Name,Lat,Lng
0,The Lodhi Hotel,28.591669,77.238131
1,PCO,28.561635,77.155590
2,Lantern's,28.643183,77.177746
3,On The Waterfront,28.592061,77.237951
4,Blue Bar,28.595403,77.170529


In [109]:
print("Restaurants information")
df_rest.head()

Restaurants information


,Name,Lat,Lng
0,Thai High,28.522127,77.181831
1,Kainoosh,28.542631,77.156886
2,Olive Bar & Kitchen,28.526103,77.183976
3,The Big Chill Cafe,28.528201,77.217748
4,Maxims,28.552849,77.242065


In [110]:
#setting up k-means
k_meansR = KMeans(init="k-means++", n_clusters=11, n_init=12)
k_meansR.fit(df_rest[['Lat','Lng']])
k_meansR_labels = k_meansR.labels_
k_meansR_labels

array([ 8,  0,  8,  4,  4,  1,  4,  1,  0,  7,  6,  1,  1,  7,  2,  7,  4,
        1,  3,  6,  1,  0,  1,  2, 10,  2,  5,  9,  5,  5,  3,  1,  2,  2,
        1,  0,  4,  4,  6,  7,  0,  4,  1,  1,  0,  5,  5,  1,  3,  1,  2,
        4,  0,  0, 10,  8,  4,  4,  7, 10,  2,  8,  1,  5,  2,  2,  7,  1,
        7,  5,  5,  1,  2,  1,  5,  4,  2,  5,  4,  2,  1,  5, 10,  2,  0,
        3,  2,  1,  4, 10,  2,  2,  2, 10,  0,  4,  2,  6,  2],
      dtype=int32)

In [111]:
k_meansR_cluster_centers = k_meansR.cluster_centers_
k_meansR_cluster_centers

array([[28.54311762, 77.1458237 ],
       [28.59276341, 77.22848013],
       [28.63150408, 77.22025563],
       [28.64408003, 77.1052639 ],
       [28.53434813, 77.22991047],
       [28.55774556, 77.19694329],
       [28.69719106, 77.19412857],
       [28.59211079, 77.18385736],
       [28.51427588, 77.18857082],
       [28.65731067, 77.3170981 ],
       [28.5577109 , 77.26484911]])

In [112]:
#analogue for bars
k_meansB = KMeans(init="k-means++", n_clusters=11, n_init=12)
k_meansB.fit(df_bars[['Lat','Lng']])
k_meansB_labels = k_meansB.labels_
k_meansB_cluster_centers = k_meansB.cluster_centers_

In [113]:
#analogue for bars
k_meansM = KMeans(init="k-means++", n_clusters=11, n_init=12)
k_meansM.fit(df_metro[['Lat','Lng']])
k_meansM_labels = k_meansM.labels_
k_meansM_cluster_centers = k_meansM.cluster_centers_

In [114]:
#append kmeans cluster number to dafa frame for easy colouring
df_rest['Cluster Nr._r'] = k_meansR_labels
df_bars['Cluster Nr._b'] = k_meansB_labels
df_metro['Cluster Nr._m'] = k_meansM_labels

In [115]:
df_metro.head()

,Name,Lat,Lng,Cluster Nr._m
0,Delhi Aerocity Metro Station,28.548572,77.120876,2
1,Dwarka Metro Station,28.614938,77.022775,7
2,New Delhi | नई दिल्ली Metro Station,28.642803,77.223351,4
3,Patel Chowk Metro Station,28.622511,77.213679,4
4,Indira Gandhi International Airport Metro Station,28.555696,77.086223,2


In [124]:
#plotting map
address = 'Delhi'
geolocator = Nominatim(user_agent="delhi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#setting cluster number as 6
k=6

# create map
MyMap = folium.Map(location=[latitude, longitude], zoom_start=11)

# setting color scheme for the clusters (R)
X = np.arange(k)
Ys = [i + X + (i*X)**2 for i in range(k)]
colors_array = cm.Greens(np.linspace(0, 1, len(Ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map (R)
markers_colors = []
for lat, lng, counter in zip(df_rest['Lat'], df_rest['Lng'], df_rest['Cluster Nr._r']):
    label = folium.Popup(str(df_rest['Cluster Nr._r']) + ' Cluster ' + str(counter), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color= rainbow[counter-5],
        fill=True,
        fill_color=rainbow[counter-5],
        fill_opacity=0.7).add_to(MyMap)
 

# setting color scheme for the clusters (B)
X2 = np.arange(k)
Ys2 = [i + X2 + (i*X2)**2 for i in range(k)]
colors_array = cm.Reds(np.linspace(0, 1, len(Ys2)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map (B)
markers_colors2 = []
for lat, lng, counter in zip(df_bars['Lat'], df_bars['Lng'], df_bars['Cluster Nr._b']):
    label = folium.Popup(str(df_bars['Cluster Nr._b']) + ' Cluster ' + str(counter), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[counter-5],
        fill=True,
        fill_color=rainbow[counter-5],
        fill_opacity=0.7).add_to(MyMap)

# setting color scheme for the clusters (M)
X3 = np.arange(k)
Ys3 = [i + X3 + (i*X3)**2 for i in range(k)]
colors_array = cm.Blues(np.linspace(0, 1, len(Ys2)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map (B)
markers_colors3 = []
for lat, lng, counter in zip(df_metro['Lat'], df_metro['Lng'], df_metro['Cluster Nr._m']):
    label = folium.Popup(str(df_metro['Cluster Nr._m']) + ' Cluster ' + str(counter), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[counter-5],
        fill=True,
        fill_color=rainbow[counter-5],
        fill_opacity=0.7).add_to(MyMap)

In [125]:
MyMap

In [126]:
tooltip = "Click me!"

#cluster centers

folium.Marker(
    [28.69719106, 77.19412857], popup="<i>Potential Location 1</i>", tooltip=tooltip
).add_to(MyMap)

folium.Marker(
    [28.65731067, 77.3170981], popup="<i>Potential Location 2</i>", tooltip=tooltip
).add_to(MyMap)

In [127]:
MyMap